Libraries and modules

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import glob

from xdas.io.febus import read as read_das
from xdas.io.febus import correct_gps_time

Pathes

In [2]:
user = 'ipgp'

if user == 'ipgp':
    #path1 = '/localstorage/hugonnet/DAS_2022/SR-STROMBOLI-2022-09-AQ08-FIBERGREEN/' #path to h5-file 
    path1 = '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/'
    path2 = '/media/hugonnet/DISKENOIR/fichiers-nc/' # path for new nc-file
    path3 = '/net/runic/moby/data/projets/monidas/biagioli/Geolocation_DAS/2022/' # path to geolocation information of the fiber

elif user == 'eleonore' :
    path1 = '/Users/stutz/DAS/DAS_STROMBOLI/2022_STROMBOLI_MANIP/SILIXA_iDAS/DATA_VLP_FEBUS/' #path to h5-file 
    path2 = path1 # path for new nc-file
    path3 = path1 # path to geolocation information of the fiber
    
elif user == 'mba' :
    path1 = '' #path to h5-file 
    path2 = '' # path for new nc-file
    path3 = '' # path to geolocation information of the fiber

Geolocation infomation of the fiber

In [3]:
fibre =  pd.read_excel(path3+'DAS_channels_2022_UTM.xlsx')
fibre

,Unnamed: 0,Lon,Lat,Alt,orientation,channel,offset
0,0,520007.664573,4.294532e+06,201.869301,258.496775,56,112
1,1,520005.809986,4.294531e+06,202.385264,259.523808,57,114
2,2,520003.990515,4.294531e+06,202.910522,260.460538,58,116
3,3,520002.169435,4.294531e+06,203.691057,263.081228,59,118
4,4,520000.634709,4.294530e+06,204.896641,263.690817,60,120
...,...,...,...,...,...,...,...
2765,2765,519224.120264,4.294316e+06,531.965556,61.905754,2821,5642
2766,2766,519225.772891,4.294317e+06,531.215331,61.905766,2822,5644
2767,2767,519227.425517,4.294318e+06,530.465105,61.905775,2823,5646
2768,2768,519229.078144,4.294319e+06,529.714879,61.905781,2824,5648


In [4]:
mid_long = fibre['Lon'].values
mid_lat = fibre['Lat'].values
mid_alt = fibre['Alt'].values
mid_ori = fibre['orientation'].values
mid_ch = fibre['channel'].values
mid_offset = fibre['offset'].values

In [5]:
beg_ch = np.arange(0, 56)
end_ch = np.arange(2826,3250)
channels = np.concatenate((beg_ch, mid_ch, end_ch))

In [6]:
beg_nan = np.full(56, np.nan)
end_nan = np.full(424, np.nan)

longitude = np.concatenate((beg_nan, mid_long, end_nan))
latitude = np.concatenate((beg_nan, mid_lat, end_nan))
altitude = np.concatenate((beg_nan, mid_alt, end_nan))
orientation = np.concatenate((beg_nan, mid_ori, end_nan))

Reading of the .h5 files into datasets and conversion in .nc

In [28]:
files = glob.glob(path1 + "*2022-11-03*.h5")
files.sort()
files

['/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_11-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_10-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_12-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_13-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_16-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_2023/SR-STROMBOLI-2022-10-AQ14-FIBERGREEN/SR_DS_STROMBOLI_2022-11-03_17-12-11_UTC.h5',
 '/net/runic/moby/data/projets/monidas/stromboli/stromboli_2023/das_20

In [30]:
len(files)

24

In [31]:
i = 1

for file in files :

    xdas = read_das(file, preprocess=correct_gps_time)

    dimensions = xdas.dims
    strain_rates = xdas.values
    das = xdas.coords
    times_extended = pd.to_datetime(das['time'])

    ds = xr.Dataset({
        'strain_rates': (dimensions, strain_rates),
        'longitude': ('channels', longitude),
        'latitude': ('channels', latitude),
        'altitude': ('channels', altitude),
        'orientation': ('channels', orientation)
            })
    ds['time'] = times_extended
    ds['offset'] = das['offset']
    ds['channels'] = channels

    ds.attrs['unit'] = f'strain rates in 10e-8 /s'
    ds.attrs['gauge_length'] = '5m'

    ds = ds.rename({'time': 'times', 'offset': 'offsets'})

    fileout = file.replace('.h5','.nc').replace(path1, path2).replace('SR_DS','SR_FEBUS')
    ds.to_netcdf(fileout)

    print(f'file {i} is done')
    i +=1

/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds['time'] = times_extended
/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahe

file 1 is done


/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds['time'] = times_extended
/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahe

file 2 is done


/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds['time'] = times_extended
/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahe

...

file 22 is done


/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds['time'] = times_extended
/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahe

file 23 is done


/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds['time'] = times_extended
/tmp/ipykernel_2492/629128973.py:24: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahe

file 24 is done
